In [1]:
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-yolo5-project\\reseach'

In [2]:
import os 

os.chdir("../")
%pwd

'c:\\Users\\44787\\Desktop\\projects\\end-to-end-yolo5-project'

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainConfig:
    root_dir: Path
    yolo5_rep_path: Path
    defined_yaml_file: Path
    MODEL_TRAINER_PRETRAINED_WEIGHT_NAME: str

In [7]:
from yolo5.logger import logging
from yolo5.constant import *
from yolo5.entity.config_entity import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(self, config=CONFIG_PATH):
        self.config = read_yaml(config)
        
    def get_model_train_config(self)-> ModelTrainConfig:
        config = self.config.model_training
        create_directories([config.root_dir])
        
        model_train_config = ModelTrainConfig(
            root_dir = config.root_dir,
            yolo5_rep_path=config.yolo5_rep_path,
            defined_yaml_file=config.defined_yaml_file,
            MODEL_TRAINER_PRETRAINED_WEIGHT_NAME=config.MODEL_TRAINER_PRETRAINED_WEIGHT_NAME,
        )
        return model_train_config

In [8]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...


In [53]:
from yolo5.exception import AppException
import os
import sys
import yaml

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [76]:
class ModelTraining:
    def __init__(self, config:ModelTrainConfig):
        self.config = config
    
    def mode_train(self):
        logging.info("clone the yolo5 repository")
        # os.system(f"git clone {self.config.yolo5_rep_path}")
        
        
        with open(self.config.defined_yaml_file, 'r') as stream:
            num_classes = str(yaml.safe_load(stream)['nc'])
            
        
        with open('yolov5/models/yolov5s.yaml', "rb") as yaml_file:
            logging.info("Read yaml file successfully")
            model_yaml =  yaml.safe_load(yaml_file)
            
        # print(model_yaml)
        model_yaml['nc'] = int(num_classes)
        # print(model_yaml)
        
        print(self.config.MODEL_TRAINER_PRETRAINED_WEIGHT_NAME)
        with open(f'yolov5/models/custom_{self.config.MODEL_TRAINER_PRETRAINED_WEIGHT_NAME}.yaml', 'w') as f:
                yaml.dump(model_yaml, f)
                
                
        batch_size = 16
        no_epochs = 1
        weight_name = "yolov5s.pt"
                
        # os.system(f"cd yolov5/ && python train.py --img 416 --batch {batch_size} --epochs {no_epochs} --data ../data.yaml --cfg ./models/custom_yolov5s.yaml --weights {weight_name} --name yolov5s_results  --cache")
        os.system(f"python train.py --img 416 --batch {batch_size} --epochs {no_epochs} --data ../data.yaml --cfg ./models/custom_yolov5s.yaml --weights {weight_name} --name yolov5s_results  --cache")
        
        os.system("cp yolov5/runs/train/yolov5s_results/weights/best.pt yolov5/")
        os.makedirs(self.config.root_dir, exist_ok=True)
        os.system(f"cp yolov5/runs/train/yolov5s_results/weights/best.pt {self.config.root_dir}/")
           
        os.system("rm -rf yolov5/runs")
    
        

In [77]:
try:
    config = ConfigurationManager()
    model_train_config = config.get_model_train_config()
    model = ModelTraining(model_train_config)
    model.mode_train()
except Exception as e:
    raise AppException(e, sys)

yolov5s
